In [2]:
from underthesea import sent_tokenize
from langchain_text_splitters import CharacterTextSplitter
from typing import List, Dict
import json
import os
from tqdm import tqdm  # Nhập tqdm

class VietnameseTextProcessor:
    def __init__(self, max_length: int = 1000, overlap: int = 200):
        self.max_length = max_length
        self.overlap = overlap
        
    def read_text_file(self, file_path: str) -> str:
        """Đọc file văn bản với encoding utf-8"""
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    
    def preprocess_text(self, text: str) -> str:
        """Tiền xử lý văn bản: giữ nguyên định dạng và dấu câu"""
        text = text.replace('\r\n', '\n')
        text = '\n'.join(line.strip() for line in text.split('\n'))
        return text
    
    def split_into_sentences(self, text: str) -> List[str]:
        """Tách văn bản thành các câu sử dụng Underthesea"""
        sentences = sent_tokenize(text)
        valid_sentences = [s.strip() for s in sentences if len(s.split()) > 5]
        return valid_sentences
    
    def create_chunks(self, sentences: List[str]) -> List[str]:
        """Ghép các câu thành chunk sử dụng LangChain"""
        splitter = CharacterTextSplitter(
            chunk_size=self.max_length,
            chunk_overlap=self.overlap,
            separator="\n",
            length_function=len
        )
        text = '\n'.join(sentences)
        chunks = splitter.split_text(text)
        return chunks
    
    def create_alpaca_format(self, chunks: List[str], author: str) -> List[Dict]:
        """Tạo format Alpaca từ các chunk"""
        dataset = []
        for i in range(len(chunks)-1):
            sample = {
                "instruction": f"Hãy viết tiếp đoạn văn sau theo phong cách của {author}",
                "input": chunks[i],
                "output": chunks[i+1]
            }
            dataset.append(sample)
        return dataset
    
    def save_dataset(self, dataset: List[Dict], output_file: str):
        """Lưu dataset dưới dạng JSON"""
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(dataset, f, ensure_ascii=False, indent=2)

    def process_folder(self, input_folder: str, author: str, output_file: str):
        """Xử lý toàn bộ quy trình cho tất cả các file trong thư mục"""
        all_datasets = []
        
        # Lặp qua tất cả các file trong thư mục với tqdm để hiển thị tiến trình
        files = [f for f in os.listdir(input_folder) if f.endswith('.txt')]
        
        with tqdm(total=len(files), desc="Processing files") as pbar:
            for filename in files:
                file_path = os.path.join(input_folder, filename)
                
                # Đọc và tiền xử lý văn bản
                text = self.read_text_file(file_path)
                text = self.preprocess_text(text)
                
                # Tách câu và tạo chunks
                sentences = self.split_into_sentences(text)
                chunks = self.create_chunks(sentences)
                
                # Tạo dataset cho từng file
                dataset = self.create_alpaca_format(chunks, author)
                all_datasets.extend(dataset)  # Gộp vào danh sách tổng
                
                # Cập nhật tiến trình với tên file hiện tại
                pbar.set_postfix_str(filename)
                pbar.update(1)  # Cập nhật tiến trình

        # Lưu tất cả dataset vào file đầu ra
        self.save_dataset(all_datasets, output_file)
        
        return len(all_datasets)

def main():
    processor = VietnameseTextProcessor(
        max_length=1500,
        overlap=200
    )
    
    input_folder = "data-raw/Thich Nhat Hanh"  # Thư mục chứa các file văn bản
    author = "Thích Nhất Hạnh"  # Tên tác giả
    output_file = "dataset.json" # File dataset đầu ra
    
    num_samples = processor.process_folder(input_folder, author, output_file)
    
    print(f"Đã xử lý xong:")
    print(f"- Số lượng mẫu trong dataset: {num_samples}")
    print(f"- Dataset được lưu tại: {output_file}")

if __name__ == "__main__":
    main()

Processing files:  71%|███████▏  | 5/7 [00:00<00:00, 42.70it/s, Di Vao Coi Tho - Bui Giang.txt]                             Created a chunk of size 2958, which is longer than the specified 1500
Created a chunk of size 1875, which is longer than the specified 1500
Created a chunk of size 2316, which is longer than the specified 1500
Created a chunk of size 1789, which is longer than the specified 1500
Created a chunk of size 1840, which is longer than the specified 1500
Created a chunk of size 2121, which is longer than the specified 1500
Created a chunk of size 1725, which is longer than the specified 1500
Created a chunk of size 1576, which is longer than the specified 1500
Processing files: 100%|██████████| 7/7 [00:00<00:00, 35.40it/s, Nha su vuong luy - To Man Thu.txt] 


Đã xử lý xong:
- Số lượng mẫu trong dataset: 1522
- Dataset được lưu tại: dataset.json
